In [79]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import os
import copy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [30]:
import pandas as pd
train = pd.read_parquet("../../Dataset/train.parquet")
valid = pd.read_parquet("../../Dataset/valid.parquet")

In [10]:
train.head(3)

,OCR,image,hero,villain,victim,other,_Hero,_Villain,_Victim,_Other,Target,All_Entities,OCR_Bert
0,Bernie Elizabeth? Be informed.Compare issues m...,covid_memes_18.png,[],[],[],"[bernie sanders, elizabeth warren]",,,,[BOTH] bernie sanders [SEP] elizabeth warren [...,[BOTH] bernie sanders [SEP] elizabeth warren [...,"[bernie sanders, elizabeth warren]",[CLS] Bernie Elizabeth? Be informed.Compare is...
1,Extending Brexit deadline October 31st order e...,covid_memes_19.png,[],[uk government],[],[],,[BVIL] uk government [EVIL],,,[BVIL] uk government [EVIL],[uk government],[CLS] Extending Brexit deadline October 31st o...
2,kwai gkwa 0964 #nnevvy applause Thais Hong Kon...,covid_memes_252.png,[thais],[],[],[hong kong],[BH] thais [EH],,,[BOTH] hong kong [EOTH],[BH] thais [EH] [BOTH] hong kong [EOTH],"[thais, hong kong]",[CLS] kwai gkwa 0964 #nnevvy applause Thais Ho...


In [44]:
# Explode dataset based on category column and merge it
def getexplodedds(ds):
    data = ds.copy()
    df = pd.DataFrame()
    for idx,col in enumerate(["hero","villain","victim","other"]):
        _df = data[["OCR",col]].explode(col)
        _df = _df[~_df[col].isna()]
        _df.columns = ["OCR","Entity"]
        _df["Target"] = idx
        _df["TargetLabel"] = col
        df = pd.concat([df,_df], axis = 0, ignore_index = True)

    return df

train_df = getexplodedds(train)
valid_df = getexplodedds(valid)

In [32]:
train_df.TargetLabel.value_counts()

other      13702
villain     2427
victim       910
hero         475
Name: TargetLabel, dtype: int64

In [45]:
# resample skewed classes
def resample(ds, target, repeat_freq):
    data = ds.copy()
    _df = data.loc[data.TargetLabel == target].copy()
    for idx in range(repeat_freq):
        data = pd.concat([data, _df], axis = 0, ignore_index=True)
    return data

train_df = resample(train_df, "hero", 10)
train_df = resample(train_df, "victim", 6)
train_df = resample(train_df, "villain", 3)

train_df.TargetLabel.value_counts()

other      13702
villain     9708
victim      6370
hero        5225
Name: TargetLabel, dtype: int64

In [46]:
train_df = train_df.sample(frac=1).reset_index(drop = True)
train_df.head(3)

,OCR,Entity,Target,TargetLabel
0,WHEN SOMEONE SAYS THEY CAN'T BELIEVE THEY HAVE...,donald trump,1,villain
1,"GARY JOHNSON THE ANTI RON PAUL u""LIBERTARIAN""",ron paul,2,victim
2,THE REPUBLICAN PARTY IS THE PARTY OF THE AMERI...,republican party,0,hero


In [47]:
# Tweet length distribution
import plotly.express as px
tweet_len = list(map(len, train_df.OCR.values))
px.histogram(tweet_len)

In [48]:
tweet_len = list(map(len, train_df.Entity.values))
px.histogram(tweet_len)

In [42]:
#train_df = train_df.head(10)

In [83]:
def getDataLoader(data, batch_size):
    # Tokenize all of the sentences and map the tokens to thier word IDs.
    input_ids_sent = []
    attention_masks_sent = []

    input_ids_ent = []
    attention_masks_ent = []

    # For every sentence...
    for sent, entity in zip(data.OCR.values, data.Entity.values):
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 200,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )

        encoded_dict_ent = tokenizer.encode_plus(
                            entity,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 20,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        # Add the encoded sentence to the list.    
        input_ids_sent.append(encoded_dict['input_ids'])
        input_ids_ent.append(encoded_dict_ent['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks_sent.append(encoded_dict['attention_mask'])
        attention_masks_ent.append(encoded_dict_ent['attention_mask'])

    # Convert the lists into tensors.
    input_ids_sent = torch.cat(input_ids_sent, dim=0)
    attention_masks_sent = torch.cat(attention_masks_sent, dim=0)

    input_ids_ent = torch.cat(input_ids_ent, dim=0)
    attention_masks_ent = torch.cat(attention_masks_ent, dim=0)
    labels = torch.tensor(data.Target.values)

    # Create the DataLoader.
    prediction_data = TensorDataset(input_ids_sent, attention_masks_sent, input_ids_ent, attention_masks_ent, labels)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler
    , batch_size=batch_size)

    return prediction_dataloader


In [84]:
# Set the batch size.  
batch_size = 32  


dataloaders = {}
dataloaders["train"] = getDataLoader(train_df,batch_size)
dataloaders["val"] = getDataLoader(valid_df,batch_size)

dataset_sizes = {}
dataset_sizes["train"] = len(train_df)
dataset_sizes["val"] = len(valid_df)

tensor([1, 2, 0,  ..., 3, 2, 3])

In [86]:
class OCRClassification(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(2,20), stride=10)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, (2,20), stride=10)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = self.pool(F.tanh(self.conv1(x)))
        x = self.pool(F.tanh(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x




In [ ]:
def train_model(model, bertmodel, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for b_input_ids_sent, b_input_mask_sent, b_input_ids_ent, b_input_mask_ent, labels in dataloaders[phase]:
                # Get output from bert model
                outputs_sent = bertmodel(b_input_ids_sent, token_type_ids=None, 
                                attention_mask=b_input_mask_sent)

                outputs_ent = bertmodel(b_input_ids_ent, token_type_ids=None, 
                                attention_mask=b_input_mask_ent)

                # Get hidden states from output
                hidden_states_sent = outputs_sent[2]
                hidden_states_ent = outputs_ent[2]

                # Stack all token embeddings from bert model
                token_embeddings_sent = torch.stack(hidden_states_sent, dim=0)
                token_embeddings_ent = torch.stack(hidden_states_ent, dim=0)

                # Take second last layer from hidden state and take a mean across tokens to get sentence level encoding
                # This will give an output of shape (batch_size * 768)
                sentence_embedding = torch.mean(token_embeddings_sent[-2,...], dim=1)
                entity_embedding = torch.mean(token_embeddings_ent[-2,...], dim=1)
                
                sentence_embedding = sentence_embedding.unsqueeze(dim = 1)
                entity_embedding = entity_embedding.unsqueeze(dim = 1)

                inputs = torch.cat([sentence_embedding, entity_embedding], dim=1).to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({
                            'epoch': epoch,
                            'model_state_dict': best_model_wts,
                            'loss': loss,
                            }, "../../Models/ocr_class.pt")

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = OCRClassification()
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

bertmodel = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
for param in bertmodel.parameters():
    param.requires_grad = False

bertmodel.eval()

model_ft = train_model(model_ft, bertmodel, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)                                 

In [57]:
#for batch in prediction_dataloader:
batch = next(iter(dataloaders["train"]))
# Add batch to GPU

# Unpack the inputs from our dataloader
b_input_ids_sent, b_input_mask_sent, b_input_ids_ent, b_input_mask_ent = batch

# Telling the model not to compute or store gradients, saving memory and 
# speeding up prediction
with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs_sent = model(b_input_ids_sent, token_type_ids=None, 
                    attention_mask=b_input_mask_sent)

    outputs_ent = model(b_input_ids_ent, token_type_ids=None, 
                    attention_mask=b_input_mask_ent)

    hidden_states_sent = outputs_sent[2]
    hidden_states_ent = outputs_ent[2]

# Concatenate the tensors for all layers. We use `stack` here to
# create a new dimension in the tensor.
token_embeddings_sent = torch.stack(hidden_states_sent, dim=0)
token_embeddings_ent = torch.stack(hidden_states_sent, dim=0)

#print(token_embeddings_sent.size())
sentence_embedding = torch.mean(token_embeddings_sent[-2,...], dim=1)
entity_embedding = torch.mean(token_embeddings_ent[-2,...], dim=1)
sentence_embedding.shape, entity_embedding.shape